### Program 4:
#### Objective:
Write a program to implement the SGD and Adagrad optimizers using the PyTorch framework, and
compare results using the MNIST digit classification dataset. Use a simple CNN to illustrate the
difference between the two optimizers.
Perform the following steps:
- Preprocess data
- Define SGD and Adagrad optimizers from scratch
- Define a simple CNN model architecture
- Train CNN model using suitable criterion and each optimizer

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [20]:

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the datasets
train_dataset = datasets.MNIST(root="./data", train=True, download=False, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=False, transform=transform)

# Create subsets
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)


In [21]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [22]:
device = torch.device('cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()

In [23]:
def train_model(num_epochs, optimizer):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            predicted = torch.argmax(output.data, dim=1)
            total_train += target.size(0)
            correct_train += (predicted == target).sum().item()
        avg_train_loss = train_loss/len(train_loader)
        train_acc = 100 * correct_train/total_train
        model.eval()
        test_loss = 0
        correct_test = 0
        total_test = 0
        with torch.no_grad():   #test loss and eval metric
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
                predicted = torch.argmax(output.data, dim=1)
                total_test += target.size(0)
                correct_test += (predicted == target).sum().item()
        avg_test_loss = test_loss/len(test_loader)
        test_acc = 100 * correct_test/total_test               #test loss and eval metric
        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc:.8f}%, 'f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.8f}%')



In [24]:
sgd = torch.optim.SGD(model.parameters(), lr=0.01)

In [25]:
train_model(5, optimizer=sgd)

Epoch 1, Train Loss: 2.3009, Train Accuracy: 12.00000000%, Test Loss: 2.2566, Test Accuracy: 22.00000000%
Epoch 2, Train Loss: 2.2699, Train Accuracy: 18.00000000%, Test Loss: 2.2307, Test Accuracy: 22.00000000%
Epoch 3, Train Loss: 2.2362, Train Accuracy: 22.50000000%, Test Loss: 2.1934, Test Accuracy: 24.00000000%
Epoch 4, Train Loss: 2.1928, Train Accuracy: 25.50000000%, Test Loss: 2.1489, Test Accuracy: 30.00000000%
Epoch 5, Train Loss: 2.1274, Train Accuracy: 35.50000000%, Test Loss: 2.0653, Test Accuracy: 46.00000000%


In [26]:
adam = torch.optim.Adagrad(model.parameters(), lr=0.01)

In [27]:
train_model(5, optimizer=adam)

Epoch 1, Train Loss: 2.1067, Train Accuracy: 40.50000000%, Test Loss: 1.0745, Test Accuracy: 64.00000000%
Epoch 2, Train Loss: 0.8090, Train Accuracy: 79.50000000%, Test Loss: 0.5518, Test Accuracy: 82.00000000%
Epoch 3, Train Loss: 0.4897, Train Accuracy: 86.50000000%, Test Loss: 0.4489, Test Accuracy: 84.00000000%
Epoch 4, Train Loss: 0.2640, Train Accuracy: 95.00000000%, Test Loss: 0.4050, Test Accuracy: 86.00000000%
Epoch 5, Train Loss: 0.2054, Train Accuracy: 93.00000000%, Test Loss: 0.3807, Test Accuracy: 88.00000000%
